# <font color = 'blue' >Turn LCID MCC MRI data into BIDS </font>
*Notebook created by Michelle Achterberg in May 2020, using the bidsify notebook of Eduard Klapwijk (https://github.com/eduardklap)* >> Original notebook can be found here: https://github.com/eur-synclab/bidsification

<u>We will use 3 steps to get out LCID MCC w01 data into BIDS</u>
1. protocolnames
2. pseudobids 
3. bids

LCID Middle Childhood Cohort (MCC) is a longitudinal dataset with 3 MRI timepoints (w1, w3, w5). For every timepoint we have participant folders with scans in par/rec format. The filenames contain the participant ID, and a number for the scan sequence (but because sometimes scans are restarted this number is not always the same for a particular sequence). It looks something like this:

/projectname/data/
                    ├── SU31_SCANS/
                    |   ├── SU31000101/
                    |   |   ├── SU31000101_4_1.PAR
                    |   |   ├── SU31000101_4_1.REC
                    |   |   ├── SU31000101_5_1.PAR
                    |   |   ├── SU31000101_5_1.REC
                    |   |   ├── SU31000101_12_1.PAR
                    |   |   └── SU31000101_12_1.REC
                    |   ├── SU31000102/
                    |   |   ├── SU31000102_3_1.PAR
                    |   |   ├── SU31000102_3_1.REC
                    |   |   ├── SU31000102_4_1.PAR
                    |   |   ├── SU31000102_4_1.REC
                    |   |   ├── SU31000102_10_1.PAR
                    |   |   └── SU31000102_10_1.REC
                    |   └── SU31034501/
                    |       ├── SU31034501_3_1.PAR
                    |       ├── SU31034501_3_1.REC
                    |       ├── SU31034501_9_1.PAR
                    |       └── SU31034501_9_1.REC
                    ├── SU33_SCANS/
                    |   ├── SU33000101/
                    |   |   ├── SU33000101_4_1.PAR
                    |   |   ├── SU33000101_4_1.REC
                    |   |   ├── SU33000101_5_1.PAR
                    |   |   ├── SU33000101_5_1.REC
                    |   |   ├── SU33000101_12_1.PAR
                    |   |   └── SU33000101_12_1.REC
                    |   ├── SU33000102/
                    |   |   ├── SU33000102_3_1.PAR
                    |   |   ├── SU33000102_3_1.REC
                    |   |   ├── SU33000102_4_1.PAR
                    |   |   ├── SU33000102_4_1.REC
                    |   |   ├── SU33000102_10_1.PAR
                    |   |   └── SU33000102_10_1.REC
                    |   └── SU31034501/
                    |       ├── SU33034501_3_1.PAR
                    |       ├── SU33034501_3_1.REC
                    |       ├── SU33034501_9_1.PAR
                    |       └── SU33034501_9_1.REC
                    └── SU35_SCANS/
                        ├── SU35000101/
                        |   ├── et cetera

## <font color = 'blue' > STEP 1: PROTOCOLNAMES </font>
The MRI files from LCID do not contain the protocolnames in the filenames, which requires an additional step before using pseudobids/bidsify. We can get the protocolinfo from the PAR files and rename the REC files accordingly.

**<font color = 'blue' >STEP 1.1: Set Directory </font>** 

For every timepoint we want to rename the files to reflect what kind of scan it is (e.g, T1-weighted, task, rsfmri). It might also be a good idea to copy the files to a new location, so that we do not have to change the original raw data.  

First we have to define the old directory (with the raw data) and the new dir  
Here it is for SU31. We can change it later and do the same for SU33 and SU35

*Note*: you have to change all the directories in the next box that have a red color to make sure that you can use this notebook. 

In [2]:
import os
import sys
import shutil
import glob
from pathlib import Path

rawDir = '/exports/fsw/Bendlab/SamenUniek/SU31_SCANS'
# it is useful to create one base directory, in which we have the raw (but renamed) data, 
                                                # and later the pseudobids and bids data
base_dir = os.chdir('/exports/fsw/Bendlab/SamenUniek/SU31_bids/')    #was '/projectname/data/bidsify/'
pseudoDir = '/exports/fsw/Bendlab/SamenUniek/SU31_bids/pseudobids'   #was '/projectname/data/bidsify/pseudobids'

**<font color = 'blue' >STEP 1.2: PAR files </font>** 

We have a code that loops through the directories and files in the root directory. 
When a file is found:  
1) its protocollines are inspected for different protocols (e.g., rsfMRI, 3DT1) in the line  
2) the file is renamed if the name does not exist yet. 

- **Note**: the protocolname in protocolline corresponds to the PAR file. You can check this by opening a PARfile in a texteditor (i.e., NEDIT).
- **Note**: the renamed protocolname (rename = etc..) should contain 4 characters


In [10]:
# check PARs: read protocolname and include in filename. 
# we will use PCGe to indicate the exclusionblock, as wave3 of LCID MCC also contains an inclusionblock 

import shutil

for root, directories, filenames in os.walk(rawDir):
    if filenames != []:
        for file in filenames:
            #extract the file name and path
            name = root + '/' + file
            name = name.replace("\\", "/")
            
            
            if file[-4:] == '.PAR':
                #open and read the protecolline needed for renaming
                with open(name, 'r') as f:
                    protocolline = f.readlines()
                
                #check what sort of file it is and setup the rename accordingly
                if 'SNAT1' in protocolline[13]:
                    rename = rawDir + '/' + root[-10:] + '/' + file[:-4] + '_' + 'SNAT1' + file[-4:]
                    print('We found a snat1 file: ' + file)
                    
                elif 'SNAT2' in protocolline[13]:
                    rename = rawDir + '/' + root[-10:] + '/' + file[:-4] + '_' + 'SNAT2' + file[-4:]
                    print('We found a snat2 file: ' + file)
                    
                elif 'SNAT3' in protocolline[13]:
                    rename = rawDir + '/' + root[-10:] + '/' + file[:-4] + '_' + 'SNAT3' + file[-4:]
                    print('We found a snat3 file: ' + file) 
                    
                elif 'PCG1' in protocolline[13]:
                    rename = rawDir + '/' + root[-10:] + '/' + file[:-4] + '_' + 'PCGe1' + file[-4:] 
                    print('We found a PCGe1 file: ' + file)
                
                elif 'PCG2' in protocolline[13]:
                    rename = rawDir + '/' + root[-10:] + '/' + file[:-4] + '_' + 'PCGe2' + file[-4:] 
                    print('We found a PCGe2 file: ' + file)
                    
                elif '3DT1' in protocolline[13]:
                    rename = rawDir + '/' + root[-10:] + '/' + file[:-4] + '_' + 'T1mri' + file[-4:]
                    print('We found a 3DT1 file: ' + file)
                    
                elif 'hires' in protocolline[13]:
                    rename = rawDir + '/' + root[-10:] + '/' + file[:-4] + '_' + 'T2str' + file[-4:]
                    print('We found a T2* file: ' + file)
                    
                elif 'B0' in protocolline[13]:
                    rename = rawDir + '/' + root[-10:] + '/' + file[:-4] + '_' + 'Bzero' + file[-4:]
                    print('We found a B0 file: ' + file)
                
                elif 'jones30_A' in protocolline[13]:
                    rename = rawDir + '/' + root[-10:] + '/' + file[:-4] + '_' + 'DTIap' + file[-4:]
                    print('We found a DTIap file: ' + file)     
                    
                elif 'jones30_P' in protocolline[13]:
                    rename = rawDir + '/' + root[-10:] + '/' + file[:-4] + '_' + 'DTIpa' + file[-4:]
                    print('We found a DTIpa file: ' + file)    
                    
                elif 'rsfMRI' in protocolline[13]:
                    rename = rawDir + '/' + root[-10:] + '/' + file[:-4] + '_' + 'rsfmr' + file[-4:]
                    print('We found a rsfmri file: ' + file)                                       
                        
                else:
                    dontRename = file[:-4]
                    continue
                    
                print(rename)
                #rename the files in the backup folder (gives warning if the file name already exists in the newDir)
                #if not os.path.isfile(rename):
                 #   os.rename(name, rename)
                    
                #else:
                 #   print('WARNING: file ' + rename + ' already exists in the folder! This file will therefore be skipped!')
                        

We found a DTIpa file: SU31049601_14_1_DTIpa_DTIpa_DTIpa_DTIpa.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_14_1_DTIpa_DTIpa_DTIpa_DTIpa_DTIpa.PAR
We found a DTIap file: SU31049601_15_1_DTIap_DTIap_DTIap_DTIap.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_15_1_DTIap_DTIap_DTIap_DTIap_DTIap.PAR
We found a snat1 file: SU31049601_3_1_SNAT1_SNAT1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_3_1_SNAT1_SNAT1_SNAT1_SNAT1_SNAT1.PAR
We found a B0 file: SU31049601_13_1_Bzero_Bzero_Bzero_Bzero.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_13_1_Bzero_Bzero_Bzero_Bzero_Bzero.PAR
We found a PCGe1 file: SU31049601_6_1_PCGe1_PCGe1_PCGe1_PCGe1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_6_1_PCGe1_PCGe1_PCGe1_PCGe1_PCGe1.PAR
We found a T2* file: SU31049601_12_1_T2str_T2str_T2str_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_12_1_T2str_T2str_T2str

We found a snat2 file: SU31011402_4_1_SNAT2_SNAT2_SNAT2_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31011402/SU31011402_4_1_SNAT2_SNAT2_SNAT2_SNAT2_SNAT2.PAR
We found a PCGe1 file: SU31011402_6_1_PCGe1_PCGe1_PCGe1_PCGe1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31011402/SU31011402_6_1_PCGe1_PCGe1_PCGe1_PCGe1_PCGe1.PAR
We found a PCGe2 file: SU31011402_7_1_PCGe2_PCGe2_PCGe2_PCGe2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31011402/SU31011402_7_1_PCGe2_PCGe2_PCGe2_PCGe2_PCGe2.PAR
We found a snat3 file: SU31011402_5_1_SNAT3_SNAT3_SNAT3_SNAT3.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31011402/SU31011402_5_1_SNAT3_SNAT3_SNAT3_SNAT3_SNAT3.PAR
We found a DTIpa file: SU31011402_11_1_DTIpa_DTIpa_DTIpa_DTIpa.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31011402/SU31011402_11_1_DTIpa_DTIpa_DTIpa_DTIpa_DTIpa.PAR
We found a T2* file: SU31011402_9_1_T2str_T2str_T2str_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31011402/SU31011402_9_1_T2str_T2str_T2str_T2

We found a snat2 file: SU31000701_4_1_SNAT2_SNAT2_SNAT2_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31000701/SU31000701_4_1_SNAT2_SNAT2_SNAT2_SNAT2_SNAT2.PAR
We found a DTIap file: SU31000701_15_1_DTIap_DTIap_DTIap_DTIap.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31000701/SU31000701_15_1_DTIap_DTIap_DTIap_DTIap_DTIap.PAR
We found a T2* file: SU31000701_12_1_T2str_T2str_T2str_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31000701/SU31000701_12_1_T2str_T2str_T2str_T2str_T2str.PAR
We found a DTIpa file: SU31000701_14_1_DTIpa_DTIpa_DTIpa_DTIpa.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31000701/SU31000701_14_1_DTIpa_DTIpa_DTIpa_DTIpa_DTIpa.PAR
We found a snat1 file: SU31000701_3_1_SNAT1_SNAT1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31000701/SU31000701_3_1_SNAT1_SNAT1_SNAT1_SNAT1_SNAT1.PAR
We found a PCGe1 file: SU31000701_7_1_PCGe1_PCGe1_PCGe1_PCGe1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31000701/SU31000701_7_1_PCGe1_PCGe1_PCGe

We found a snat3 file: SU31040501_5_1_SNAT3_SNAT3_SNAT3_SNAT3.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31040501/SU31040501_5_1_SNAT3_SNAT3_SNAT3_SNAT3_SNAT3.PAR
We found a DTIap file: SU31040501_12_1_DTIap_DTIap_DTIap_DTIap.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31040501/SU31040501_12_1_DTIap_DTIap_DTIap_DTIap_DTIap.PAR
We found a B0 file: SU31041502_10_1_Bzero_Bzero_Bzero_Bzero.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31041502/SU31041502_10_1_Bzero_Bzero_Bzero_Bzero_Bzero.PAR
We found a PCGe2 file: SU31041502_7_1_PCGe2_PCGe2_PCGe2_PCGe2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31041502/SU31041502_7_1_PCGe2_PCGe2_PCGe2_PCGe2_PCGe2.PAR
We found a 3DT1 file: SU31041502_8_1_T1mri_T1mri_T1mri_T1mri.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31041502/SU31041502_8_1_T1mri_T1mri_T1mri_T1mri_T1mri.PAR
We found a snat2 file: SU31041502_4_1_SNAT2_SNAT2_SNAT2_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31041502/SU31041502_4_1_SNAT2_SNAT2_SNAT2_SN

We found a rsfmri file: SU31015701_12_1_rsfmr_rsfmr_rsfmr_rsfmr.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31015701/SU31015701_12_1_rsfmr_rsfmr_rsfmr_rsfmr_rsfmr.PAR
We found a snat3 file: SU31015701_5_1_SNAT3_SNAT3_SNAT3_SNAT3.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31015701/SU31015701_5_1_SNAT3_SNAT3_SNAT3_SNAT3_SNAT3.PAR
We found a snat2 file: SU31015701_4_1_SNAT2_SNAT2_SNAT2_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31015701/SU31015701_4_1_SNAT2_SNAT2_SNAT2_SNAT2_SNAT2.PAR
We found a T2* file: SU31112901_12_1_T2str_T2str_T2str_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31112901/SU31112901_12_1_T2str_T2str_T2str_T2str_T2str.PAR
We found a 3DT1 file: SU31112901_11_1_T1mri_T1mri_T1mri_T1mri.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31112901/SU31112901_11_1_T1mri_T1mri_T1mri_T1mri_T1mri.PAR
We found a B0 file: SU31112901_13_1_Bzero_Bzero_Bzero_Bzero.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31112901/SU31112901_13_1_Bzero_Bzero_Bzero

We found a PCGe1 file: SU31064801_6_1_PCGe1_PCGe1_PCGe1_PCGe1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31064801/SU31064801_6_1_PCGe1_PCGe1_PCGe1_PCGe1_PCGe1.PAR
We found a DTIap file: SU31064801_12_1_DTIap_DTIap_DTIap_DTIap.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31064801/SU31064801_12_1_DTIap_DTIap_DTIap_DTIap_DTIap.PAR
We found a snat3 file: SU31064801_5_1_SNAT3_SNAT3_SNAT3_SNAT3.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31064801/SU31064801_5_1_SNAT3_SNAT3_SNAT3_SNAT3_SNAT3.PAR
We found a DTIpa file: SU31064801_11_1_DTIpa_DTIpa_DTIpa_DTIpa.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31064801/SU31064801_11_1_DTIpa_DTIpa_DTIpa_DTIpa_DTIpa.PAR
We found a 3DT1 file: SU31064801_8_1_T1mri_T1mri_T1mri_T1mri.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31064801/SU31064801_8_1_T1mri_T1mri_T1mri_T1mri_T1mri.PAR
We found a DTIap file: SU31076001_15_1_DTIap_DTIap_DTIap_DTIap.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31076001/SU31076001_15_1_DTIap_DTIap_DTI

KeyboardInterrupt: 

In [16]:
for root, directories, filenames in os.walk(rawDir):
    if filenames != []:
        for file in filenames:
            #extract the file name and path
            name = root + '/' + file
            name = name.replace("\\", "/")
            

                #check what sort of file it is and setup the rename accordingly
            if 'SNAT1_SNAT1_SNAT1_SNAT1' in name:
                #rename = rawDir + '/' + root[-10:] + '/' + file[:-4] + '_' + 'SNAT1' + file[-4:]
                print('We found a snat1 file: ' + file)
                rename = name.replace('SNAT1_SNAT1_SNAT1_SNAT1','SNAT1')
                print(rename)
                os.rename(name,rename)

We found a snat1 file: SU31049601_3_1_SNAT1_SNAT1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_3_1_SNAT1.PAR
We found a snat1 file: SU31066801_3_1_SNAT1_SNAT1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31066801/SU31066801_3_1_SNAT1.PAR
We found a snat1 file: SU31064602_3_1_SNAT1_SNAT1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31064602/SU31064602_3_1_SNAT1.PAR
We found a snat1 file: SU31012501_3_1_SNAT1_SNAT1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31012501/SU31012501_3_1_SNAT1.PAR
We found a snat1 file: SU31026502_3_1_SNAT1_SNAT1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31026502/SU31026502_3_1_SNAT1.PAR
We found a snat1 file: SU31011402_3_1_SNAT1_SNAT1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31011402/SU31011402_3_1_SNAT1.PAR
We found a snat1 file: SU31004502_3_1_SNAT1_SNAT1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31004502/SU31004502

In [11]:
for root, directories, filenames in os.walk(rawDir):
    if filenames != []:
        for file in filenames:
            #extract the file name and path
            name = root + '/' + file
            name = name.replace("\\", "/")
            

                #check what sort of file it is and setup the rename accordingly
            if 'SNAT1_SNAT1' in name:
                #rename = rawDir + '/' + root[-10:] + '/' + file[:-4] + '_' + 'SNAT1' + file[-4:]
                print('We found a snat1 file: ' + file)
                rename = name.replace('SNAT1_SNAT1','SNAT1')
                print(rename)
            
            elif 'SNAT2_SNAT2' in name:
                print('We found a snat2 file: ' + file)
                rename = name.replace('SNAT2_SNAT2','SNAT2')
                print(rename)
            
            elif 'SNAT3_SNAT3' in name:
                print('We found a snat3 file: ' + file)
                rename = name.replace('SNAT3_SNAT3','SNAT3')
                print(rename)

            elif 'PCGe1_PCGe1' in name:
                print('We found a PCGe1 file: ' + file)
                rename = name.replace('PCGe1_PCGe1','PCGe1')
                print(rename)

            elif 'PCGe2_PCGe2' in name:
                print('We found a PCGe2 file: ' + file)
                rename = name.replace('PCGe2_PCGe2','PCGe2')
                print(rename)
                
            elif 'T1mri_T1mri' in name:
                print('We found a T1mri file: ' + file)
                rename = name.replace('T1mri_T1mri','T1mri')
                print(rename)
                
            elif 'T2str_T2str' in name:
                print('We found a T2str file: ' + file)
                rename = name.replace('T2str_T2str','T2str')
                print(rename)       

            elif 'Bzero_Bzero' in name:
                print('We found a Bzero file: ' + file)
                rename = name.replace('Bzero_Bzero','Bzero')
                print(rename)
                
            elif 'DTIap_DTIap' in name:
                print('We found a DTIap file: ' + file)
                rename = name.replace('DTIap_DTIap','DTIap')
                print(rename)

            elif 'DTIpa_DTIpa' in name:
                print('We found a DTIpa file: ' + file)
                rename = name.replace('DTIpa_DTIpa','DTIpa')
                print(rename)   
                    
            elif 'rsfmr_rsfmr' in name:
                print('We found a rsfmr file: ' + file)
                rename = name.replace('rsfmr_rsfmr','rsfmr')
                print(rename)                                       
                        
            else:
                dontRename = file[:-4]
                continue
                    
                #rename the files in the backup folder (gives warning if the file name already exists in the newDir)
            if not os.path.isfile(rename):
                    os.rename(name, rename)
                    
            else:
                print('WARNING: file ' + rename + ' already exists in the folder! This file will therefore be skipped!')
                        

We found a snat3 file: SU31049601_5_1_SNAT3_SNAT3.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_5_1_SNAT3.PAR
We found a Bzero file: SU31049601_13_1_Bzero_Bzero.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_13_1_Bzero.PAR
We found a PCGe2 file: SU31049601_7_1_PCGe2_PCGe2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_7_1_PCGe2.PAR
We found a DTIpa file: SU31049601_14_1_DTIpa_DTIpa.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_14_1_DTIpa.PAR
We found a T2str file: SU31049601_12_1_T2str_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_12_1_T2str.PAR
We found a rsfmr file: SU31049601_16_1_rsfmr_rsfmr.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_16_1_rsfmr.PAR
We found a DTIap file: SU31049601_15_1_DTIap_DTIap.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601/SU31049601_15_1_DTIap.PAR
We found a PCGe1 file: SU31049601_6_1_PCGe1_PCGe1.PAR
/expo

We found a T2str file: SU31028902_9_1_T2str_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31028902/SU31028902_9_1_T2str.PAR
We found a Bzero file: SU31028902_10_1_Bzero_Bzero.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31028902/SU31028902_10_1_Bzero.PAR
We found a DTIap file: SU31028902_12_1_DTIap_DTIap.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31028902/SU31028902_12_1_DTIap.PAR
We found a T1mri file: SU31028902_8_1_T1mri_T1mri.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31028902/SU31028902_8_1_T1mri.PAR
We found a PCGe2 file: SU31028902_7_1_PCGe2_PCGe2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31028902/SU31028902_7_1_PCGe2.PAR
We found a DTIpa file: SU31028902_11_1_DTIpa_DTIpa.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31028902/SU31028902_11_1_DTIpa.PAR
We found a PCGe1 file: SU31028902_6_1_PCGe1_PCGe1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31028902/SU31028902_6_1_PCGe1.PAR
We found a PCGe1 file: SU31098901_6_1_PCGe1_PCGe1.PAR
/exports/

We found a snat2 file: SU31076201_4_1_SNAT2_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31076201/SU31076201_4_1_SNAT2.PAR
We found a Bzero file: SU31076201_10_1_Bzero_Bzero.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31076201/SU31076201_10_1_Bzero.PAR
We found a T1mri file: SU31076201_8_1_T1mri_T1mri.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31076201/SU31076201_8_1_T1mri.PAR
We found a DTIpa file: SU31076201_11_1_DTIpa_DTIpa.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31076201/SU31076201_11_1_DTIpa.PAR
We found a PCGe1 file: SU31076201_6_1_PCGe1_PCGe1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31076201/SU31076201_6_1_PCGe1.PAR
We found a rsfmr file: SU31076201_13_1_rsfmr_rsfmr.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31076201/SU31076201_13_1_rsfmr.PAR
We found a snat3 file: SU31076201_5_1_SNAT3_SNAT3.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31076201/SU31076201_5_1_SNAT3.PAR
We found a T2str file: SU31076201_9_1_T2str_T2str.PAR
/exports/

We found a PCGe2 file: SU31112901_16_1_PCGe2_PCGe2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31112901/SU31112901_16_1_PCGe2.PAR
We found a Bzero file: SU31112901_13_1_Bzero_Bzero.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31112901/SU31112901_13_1_Bzero.PAR
We found a snat2 file: SU31112901_4_1_SNAT2_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31112901/SU31112901_4_1_SNAT2.PAR
We found a snat3 file: SU31112901_8_1_SNAT3_SNAT3.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31112901/SU31112901_8_1_SNAT3.PAR
We found a T2str file: SU31112901_12_1_T2str_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31112901/SU31112901_12_1_T2str.PAR
We found a PCGe1 file: SU31112901_14_1_PCGe1_PCGe1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31112901/SU31112901_14_1_PCGe1.PAR
We found a T1mri file: SU31112901_11_1_T1mri_T1mri.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31112901/SU31112901_11_1_T1mri.PAR
We found a DTIap file: SU31112901_18_1_DTIap_DTIap.PAR
/exp

We found a T1mri file: SU31017601_8_1_T1mri_T1mri.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31017601/SU31017601_8_1_T1mri.PAR
We found a snat2 file: SU31017601_4_1_SNAT2_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31017601/SU31017601_4_1_SNAT2.PAR
We found a PCGe2 file: SU31017601_7_1_PCGe2_PCGe2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31017601/SU31017601_7_1_PCGe2.PAR
We found a DTIap file: SU31017601_12_1_DTIap_DTIap.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31017601/SU31017601_12_1_DTIap.PAR
We found a snat2 file: SU31013302_4_1_SNAT2_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31013302/SU31013302_4_1_SNAT2.PAR
We found a T2str file: SU31013302_9_1_T2str_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31013302/SU31013302_9_1_T2str.PAR
We found a PCGe2 file: SU31013302_7_1_PCGe2_PCGe2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31013302/SU31013302_7_1_PCGe2.PAR
We found a Bzero file: SU31013302_10_1_Bzero_Bzero.PAR
/exports/fsw

...

We found a rsfmr file: SU31037802_13_1_rsfmr_rsfmr.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31037802/SU31037802_13_1_rsfmr.PAR
We found a DTIpa file: SU31037802_11_1_DTIpa_DTIpa.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31037802/SU31037802_11_1_DTIpa.PAR
We found a snat1 file: SU31037802_3_1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31037802/SU31037802_3_1_SNAT1.PAR
We found a PCGe1 file: SU31037802_6_1_PCGe1_PCGe1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31037802/SU31037802_6_1_PCGe1.PAR
We found a snat3 file: SU31037802_5_1_SNAT3_SNAT3.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31037802/SU31037802_5_1_SNAT3.PAR
We found a T2str file: SU31037802_9_1_T2str_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31037802/SU31037802_9_1_T2str.PAR
We found a PCGe1 file: SU31070202_7_1_PCGe1_PCGe1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31070202/SU31070202_7_1_PCGe1.PAR
We found a snat2 file: SU31070202_5_1_SNAT2_SNAT2.PAR
/exports/fs

We found a PCGe1 file: SU31071801_6_1_PCGe1_PCGe1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31071801/SU31071801_6_1_PCGe1.PAR
We found a snat1 file: SU31071801_3_1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31071801/SU31071801_3_1_SNAT1.PAR
We found a DTIpa file: SU31071801_11_1_DTIpa_DTIpa.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31071801/SU31071801_11_1_DTIpa.PAR
We found a snat2 file: SU31071801_4_1_SNAT2_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31071801/SU31071801_4_1_SNAT2.PAR
We found a T2str file: SU31071801_9_1_T2str_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31071801/SU31071801_9_1_T2str.PAR
We found a T1mri file: SU31071801_8_1_T1mri_T1mri.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31071801/SU31071801_8_1_T1mri.PAR
We found a snat3 file: SU31071801_5_1_SNAT3_SNAT3.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31071801/SU31071801_5_1_SNAT3.PAR
We found a rsfmr file: SU31071801_13_1_rsfmr_rsfmr.PAR
/exports/fsw

We found a Bzero file: SU31024202_14_1_Bzero_Bzero.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31024202/SU31024202_14_1_Bzero.PAR
We found a T1mri file: SU31024202_12_1_T1mri_T1mri.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31024202/SU31024202_12_1_T1mri.PAR
We found a T2str file: SU31074802_14_1_T2str_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31074802/SU31074802_14_1_T2str.PAR
We found a snat3 file: SU31074802_5_1_SNAT3_SNAT3.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31074802/SU31074802_5_1_SNAT3.PAR
We found a snat1 file: SU31074802_3_1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31074802/SU31074802_3_1_SNAT1.PAR
We found a snat2 file: SU31074802_4_1_SNAT2_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31074802/SU31074802_4_1_SNAT2.PAR
We found a rsfmr file: SU31074802_18_1_rsfmr_rsfmr.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31074802/SU31074802_18_1_rsfmr.PAR
We found a T1mri file: SU31074802_13_1_T1mri_T1mri.PAR
/expor

We found a DTIap file: SU31087201_14_1_DTIap_DTIap.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31087201/SU31087201_14_1_DTIap.PAR
We found a snat2 file: SU31087201_4_1_SNAT2_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31087201/SU31087201_4_1_SNAT2.PAR
We found a snat1 file: SU31087201_3_1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31087201/SU31087201_3_1_SNAT1.PAR
We found a Bzero file: SU31087201_12_1_Bzero_Bzero.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31087201/SU31087201_12_1_Bzero.PAR
We found a PCGe2 file: SU31087201_7_1_PCGe2_PCGe2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31087201/SU31087201_7_1_PCGe2.PAR
We found a rsfmr file: SU31087201_15_1_rsfmr_rsfmr.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31087201/SU31087201_15_1_rsfmr.PAR
We found a snat3 file: SU31087201_5_1_SNAT3_SNAT3.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31087201/SU31087201_5_1_SNAT3.PAR
We found a T1mri file: SU31087201_10_1_T1mri_T1mri.PAR
/exports

We found a snat3 file: SU31007002_5_1_SNAT3_SNAT3.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31007002/SU31007002_5_1_SNAT3.PAR
We found a Bzero file: SU31007002_10_1_Bzero_Bzero.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31007002/SU31007002_10_1_Bzero.PAR
We found a rsfmr file: SU31007002_13_1_rsfmr_rsfmr.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31007002/SU31007002_13_1_rsfmr.PAR
We found a snat1 file: SU31007002_3_1_SNAT1_SNAT1.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31007002/SU31007002_3_1_SNAT1.PAR
We found a T2str file: SU31007002_9_1_T2str_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31007002/SU31007002_9_1_T2str.PAR
We found a snat2 file: SU31007002_4_1_SNAT2_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31007002/SU31007002_4_1_SNAT2.PAR
We found a T1mri file: SU31007002_8_1_T1mri_T1mri.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31007002/SU31007002_8_1_T1mri.PAR
We found a DTIpa file: SU31007002_11_1_DTIpa_DTIpa.PAR
/exports/f

**<font color = 'blue' >STEP 1.3: REC files </font>** Next, we copy and rename the .REC files based on the corresponding .PAR files

In [6]:
#try this one first:
for root, directories, filenames in os.walk(rawDir):
    if filenames != []:
        for file in filenames:
            #extract the file name and path
            name = root + '\\' + file
            name = name.replace("\\", "/")
            
            if file[-4:] == '.PAR':
                namePAR = name
                print(namePAR)
                numberPAR = namePAR[-13:-10]
                print(numberPAR)
                sortPAR = namePAR[-9:-4]
                print(sortPAR)
                
                if sortPAR =='SNAT1':
                    nameREC = name[:-10] + '.REC'
                    print(nameREC)
                    renameREC = nameREC[:-4] + '_' + sortPAR + '.REC'
                    
                elif sortPAR =='SNAT2':
                    nameREC = name[:-10] + '.REC'
                    print(nameREC)
                    renameREC = nameREC[:-4] + '_' + sortPAR + '.REC'
                    
                elif sortPAR =='SNAT3':
                    nameREC = name[:-10] + '.REC'
                    print(nameREC)
                    renameREC = nameREC[:-4] + '_' + sortPAR + '.REC'
            
                elif sortPAR =='PCGe1':
                    nameREC = name[:-10] + '.REC'
                    print(nameREC)
                    renameREC = nameREC[:-4] + '_' + sortPAR + '.REC'
                    
                elif sortPAR =='PCGe2':
                    nameREC = name[:-10] + '.REC'
                    print(nameREC)
                    renameREC = nameREC[:-4] + '_' + sortPAR + '.REC'
                
                elif sortPAR =='T1mri':
                    nameREC = name[:-10] + '.REC'
                    print(nameREC)
                    renameREC = nameREC[:-4] + '_' + sortPAR + '.REC'
                    
                elif sortPAR =='T2str':
                    nameREC = name[:-10] + '.REC'
                    print(nameREC)
                    renameREC = nameREC[:-4] + '_' + sortPAR + '.REC'
                
                elif sortPAR =='Bzero':
                    nameREC = name[:-10] + '.REC'
                    print(nameREC)
                    renameREC = nameREC[:-4] + '_' + sortPAR + '.REC'
                    
                elif sortPAR =='DTIap':
                    nameREC = name[:-10] + '.REC'
                    print(nameREC)
                    renameREC = nameREC[:-4] + '_' + sortPAR + '.REC'
                    
                elif sortPAR =='DTIpa':
                    nameREC = name[:-10] + '.REC'
                    print(nameREC)
                    renameREC = nameREC[:-4] + '_' + sortPAR + '.REC'   
                                
                elif sortPAR =='rsfmr':
                    nameREC = name[:-10] + '.REC'
                    print(nameREC)
                    renameREC = nameREC[:-4] + '_' + sortPAR + '.REC'
                                 
                else:
                    dontRename = file[:-4]
                    continue
                
                if not os.path.isfile(renameREC):
                    print(renameREC)
                    os.rename(nameREC, renameREC)
                else:
                    print('WARNING: file ' + renameREC + ' already exists in the folder! This file will therefore be skipped!')
             
                    

/exports/fsw/Bendlab/SamenUniek/SU31_bids/pseudobids/sub-mcc113102/ses-w01lab/SU31113102_7_1_SNAT3.PAR
7_1
SNAT3
/exports/fsw/Bendlab/SamenUniek/SU31_bids/pseudobids/sub-mcc113102/ses-w01lab/SU31113102_7_1.REC
/exports/fsw/Bendlab/SamenUniek/SU31_bids/pseudobids/sub-mcc113102/ses-w01lab/SU31113102_7_1_SNAT3.REC
/exports/fsw/Bendlab/SamenUniek/SU31_bids/pseudobids/sub-mcc113102/ses-w01lab/SU31113102_4_1_SNAT2.PAR
4_1
SNAT2
/exports/fsw/Bendlab/SamenUniek/SU31_bids/pseudobids/sub-mcc113102/ses-w01lab/SU31113102_4_1.REC
/exports/fsw/Bendlab/SamenUniek/SU31_bids/pseudobids/sub-mcc113102/ses-w01lab/SU31113102_4_1_SNAT2.REC
/exports/fsw/Bendlab/SamenUniek/SU31_bids/pseudobids/sub-mcc113102/ses-w01lab/SU31113102_9_1_PCGe2.PAR
9_1
PCGe2
/exports/fsw/Bendlab/SamenUniek/SU31_bids/pseudobids/sub-mcc113102/ses-w01lab/SU31113102_9_1.REC
/exports/fsw/Bendlab/SamenUniek/SU31_bids/pseudobids/sub-mcc113102/ses-w01lab/SU31113102_9_1_PCGe2.REC
/exports/fsw/Bendlab/SamenUniek/SU31_bids/pseudobids/sub-mcc1

**<font color = 'blue' >STEP 1.4: Check </font>**  Let's have a look at the files in our newDir:
All files should now contain a protocolname in the filename and have corresponding PAR/REC files. 

In [15]:
import os
for root, directories, filenames in os.walk(rawDir):
    print(root) 
    for filename in filenames:
        print(filename)
        #break

/exports/fsw/Bendlab/SamenUniek/SU31_SCANS
Thumbs.db
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31049601
SU31049601_15_1_DTIap.REC
SU31049601_5_1_SNAT3.PAR
SU31049601_14_1_DTIpa.PAR
SU31049601_12_1_T2str.REC
SU31049601_3_1_SNAT1.PAR
SU31049601_11_1_T1mri.REC
SU31049601_4_1_SNAT2.PAR
SU31049601_11_1_T1mri.PAR
SU31049601_3_1_SNAT1.REC
SU31049601_6_1_PCGe1.PAR
SU31049601_16_1_rsfmr.PAR
SU31049601_13_1_Bzero.PAR
SU31049601_12_1_T2str.PAR
SU31049601_7_1_PCGe2.PAR
SU31049601_6_1_PCGe1.REC
SU31049601_14_1_DTIpa.REC
SU31049601_13_1_Bzero.REC
SU31049601_7_1_PCGe2.REC
SU31049601_15_1_DTIap.PAR
SU31049601_16_1_rsfmr.REC
SU31049601_4_1_SNAT2.REC
SU31049601_5_1_SNAT3.REC
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31066801
SU31066801_4_1_SNAT2.PAR
SU31066801_7_1_PCGe2.PAR
SU31066801_13_1_rsfmr.PAR
SU31066801_10_1_Bzero.REC
SU31066801_11_1_DTIpa.REC
SU31066801_8_1_T1mri.PAR
SU31066801_9_1_T2str.REC
SU31066801_7_1_PCGe2.REC
SU31066801_11_1_DTIpa.PAR
SU31066801_9_1_T2str.PAR
SU31066801_5_1_SNAT3.PA

/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31013701
SU31013701_4_1_SNAT1.REC
SU31013701_12_1_DTIpa.PAR
SU31013701_13_1_DTIap.PAR
SU31013701_9_1_T1mri.PAR
SU31013701_5_1_SNAT2.PAR
SU31013701_12_1_DTIpa.REC
SU31013701_10_1_T2str.REC
SU31013701_9_1_T1mri.REC
SU31013701_8_1_PCGe2.PAR
SU31013701_14_1_rsfmr.REC
SU31013701_13_1_DTIap.REC
SU31013701_7_1_PCGe1.PAR
SU31013701_11_1_Bzero.PAR
SU31013701_8_1_PCGe2.REC
SU31013701_4_1_SNAT1.PAR
SU31013701_7_1_PCGe1.REC
SU31013701_6_1_SNAT3.REC
SU31013701_10_1_T2str.PAR
SU31013701_14_1_rsfmr.PAR
SU31013701_11_1_Bzero.REC
SU31013701_6_1_SNAT3.PAR
SU31013701_5_1_SNAT2.REC
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31064601
SU31064601_13_1_rsfmr.PAR
SU31064601_11_1_DTIpa.PAR
SU31064601_4_1_SNAT2.PAR
SU31064601_8_1_T1mri.PAR
SU31064601_7_1_PCGe2.PAR
SU31064601_13_1_rsfmr.REC
SU31064601_9_1_T2str.REC
SU31064601_6_1_PCGe1.PAR
SU31064601_3_1_SNAT1.REC
SU31064601_6_1_PCGe1.REC
SU31064601_7_1_PCGe2.REC
SU31064601_12_1_DTIap.REC
SU31064601_10_1_Bzero.REC
SU

/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31088501
SU31088501_4_1_SNAT2.PAR
SU31088501_7_1_PCGe2.PAR
SU31088501_6_1_PCGe1.PAR
SU31088501_3_1_SNAT1.PAR
SU31088501_12_1_DTIap.REC
SU31088501_13_1_rsfmr.REC
SU31088501_3_1_SNAT1.REC
SU31088501_8_1_T1mri.PAR
SU31088501_4_1_SNAT2.REC
SU31088501_10_1_Bzero.PAR
SU31088501_7_1_PCGe2.REC
SU31088501_11_1_DTIpa.REC
SU31088501_10_1_Bzero.REC
SU31088501_9_1_T2str.REC
SU31088501_5_1_SNAT3.PAR
SU31088501_11_1_DTIpa.PAR
SU31088501_12_1_DTIap.PAR
SU31088501_9_1_T2str.PAR
SU31088501_6_1_PCGe1.REC
SU31088501_13_1_rsfmr.PAR
SU31088501_8_1_T1mri.REC
SU31088501_5_1_SNAT3.REC
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31080201
SU31080201_14_1_rsfmr.PAR
SU31080201_3_1_SNAT1.REC
SU31080201_10_1_T2str.REC
SU31080201_10_1_T2str.PAR
SU31080201_11_1_Bzero.PAR
SU31080201_7_1_PCGe1.REC
SU31080201_9_1_T1mri.REC
SU31080201_3_1_SNAT1.PAR
SU31080201_6_1_SNAT3.PAR
SU31080201_8_1_PCGe2.PAR
SU31080201_9_1_T1mri.PAR
SU31080201_4_1_SNAT2.PAR
SU31080201_13_1_DTIap.PAR
SU31

/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31098102
SU31098102_12_1_DTIpa.PAR
SU31098102_11_1_T1mri.REC
SU31098102_14_1_rsfmr.REC
SU31098102_12_1_DTIpa.REC
SU31098102_14_1_rsfmr.PAR
SU31098102_3_1_SNAT1.PAR
SU31098102_13_1_DTIap.REC
SU31098102_6_1_PCGe1.PAR
SU31098102_4_1_SNAT2.PAR
SU31098102_5_1_SNAT3.PAR
SU31098102_4_1_SNAT2.REC
SU31098102_10_1_Bzero.PAR
SU31098102_13_1_DTIap.PAR
SU31098102_7_1_PCGe2.REC
SU31098102_5_1_SNAT3.REC
SU31098102_9_1_T2str.REC
SU31098102_7_1_PCGe2.PAR
SU31098102_10_1_Bzero.REC
SU31098102_6_1_PCGe1.REC
SU31098102_3_1_SNAT1.REC
SU31098102_11_1_T1mri.PAR
SU31098102_9_1_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31017402
SU31017402_4_1_SNAT2.PAR
SU31017402_10_1_Bzero.REC
SU31017402_13_1_rsfmr.PAR
SU31017402_5_1_SNAT3.REC
SU31017402_9_1_T2str.PAR
SU31017402_13_1_rsfmr.REC
SU31017402_6_1_PCGe1.REC
SU31017402_3_1_SNAT1.REC
SU31017402_12_1_DTIap.PAR
SU31017402_4_1_SNAT2.REC
SU31017402_10_1_Bzero.PAR
SU31017402_7_1_PCGe2.PAR
SU31017402_6_1_PCGe1.PAR
SU

/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31110502
SU31110502_7_1_PCGe1.REC
SU31110502_3_1_SNAT1.PAR
SU31110502_6_1_SNAT3.REC
SU31110502_4_1_SNAT2.PAR
SU31110502_12_1_DTIpa.PAR
SU31110502_14_1_rsfmr.REC
SU31110502_3_1_SNAT1.REC
SU31110502_6_1_SNAT3.PAR
SU31110502_8_1_PCGe2.PAR
SU31110502_13_1_DTIap.REC
SU31110502_14_1_rsfmr.PAR
SU31110502_7_1_PCGe1.PAR
SU31110502_10_1_T2str.PAR
SU31110502_13_1_DTIap.PAR
SU31110502_12_1_DTIpa.REC
SU31110502_8_1_PCGe2.REC
SU31110502_11_1_Bzero.REC
SU31110502_9_1_T1mri.PAR
SU31110502_10_1_T2str.REC
SU31110502_11_1_Bzero.PAR
SU31110502_4_1_SNAT2.REC
SU31110502_9_1_T1mri.REC
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31090101
SU31090101_11_1_PCGe1.PAR
SU31090101_7_1_SNAT2.PAR
SU31090101_11_1_PCGe1.REC
SU31090101_8_1.REC
SU31090101_3_1_SNAT1.PAR
SU31090101_3_1_SNAT1.REC
SU31090101_7_1_SNAT2.REC
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31018401
SU31018401_13_1_rsfmr.REC
SU31018401_3_1_SNAT1.PAR
SU31018401_6_1_PCGe1.REC
SU31018401_4_1_SNAT2.PAR
SU3101

...

/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31083401
SU31083401_3_1_SNAT1.PAR
SU31083401_4_1_SNAT2.REC
SU31083401_5_1_SNAT3.PAR
SU31083401_9_1_T1mri.REC
SU31083401_7_1_PCGe2.REC
SU31083401_11_1_Bzero.REC
SU31083401_6_1_PCGe1.REC
SU31083401_7_1_PCGe2.PAR
SU31083401_10_1_T2str.PAR
SU31083401_5_1_SNAT3.REC
SU31083401_11_1_Bzero.PAR
SU31083401_6_1_PCGe1.PAR
SU31083401_9_1_T1mri.PAR
SU31083401_10_1_T2str.REC
SU31083401_3_1_SNAT1.REC
SU31083401_4_1_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31018801
SU31018801_12_1_DTIap.PAR
SU31018801_4_1_SNAT2.REC
SU31018801_6_1_PCGe1.PAR
SU31018801_6_1_PCGe1.REC
SU31018801_10_1_Bzero.REC
SU31018801_8_1_T1mri.REC
SU31018801_5_1_SNAT3.REC
SU31018801_13_1_rsfmr.PAR
SU31018801_7_1_PCGe2.REC
SU31018801_9_1_T2str.PAR
SU31018801_7_1_PCGe2.PAR
SU31018801_13_1_rsfmr.REC
SU31018801_8_1_T1mri.PAR
SU31018801_9_1_T2str.REC
SU31018801_11_1_DTIpa.PAR
SU31018801_3_1_SNAT1.PAR
SU31018801_5_1_SNAT3.PAR
SU31018801_12_1_DTIap.REC
SU31018801_10_1_Bzero.PAR
SU3101

/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31076002
SU31076002_4_1_SNAT2.REC
SU31076002_10_1_Bzero.REC
SU31076002_12_1_DTIap.PAR
SU31076002_12_1_DTIap.REC
SU31076002_4_1_SNAT2.PAR
SU31076002_11_1_DTIpa.REC
SU31076002_5_1_SNAT3.REC
SU31076002_6_1_PCGe1.REC
SU31076002_3_1_SNAT1.REC
SU31076002_5_1_SNAT3.PAR
SU31076002_13_1_rsfmr.PAR
SU31076002_11_1_DTIpa.PAR
SU31076002_9_1_T2str.REC
SU31076002_7_1_PCGe2.PAR
SU31076002_6_1_PCGe1.PAR
SU31076002_8_1_T1mri.REC
SU31076002_7_1_PCGe2.REC
SU31076002_3_1_SNAT1.PAR
SU31076002_13_1_rsfmr.REC
SU31076002_8_1_T1mri.PAR
SU31076002_10_1_Bzero.PAR
SU31076002_9_1_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31094801
SU31094801_4_1_SNAT2.PAR
SU31094801_9_1_T2str.REC
SU31094801_5_1_SNAT3.PAR
SU31094801_8_1_T1mri.REC
SU31094801_6_1_PCGe1.PAR
SU31094801_4_1_SNAT2.REC
SU31094801_6_1_PCGe1.REC
SU31094801_5_1_SNAT3.REC
SU31094801_7_1_PCGe2.PAR
SU31094801_3_1_SNAT1.REC
SU31094801_8_1_T1mri.PAR
SU31094801_7_1_PCGe2.REC
SU31094801_10_1_Bzero.REC
SU310948

/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31040201
SU31040201_10_1_Bzero.PAR
SU31040201_6_1_PCGe1.PAR
SU31040201_10_1_Bzero.REC
SU31040201_3_1_SNAT1.REC
SU31040201_12_1_DTIap.REC
SU31040201_3_1_SNAT1.PAR
SU31040201_8_1_T1mri.PAR
SU31040201_8_1_T1mri.REC
SU31040201_4_1_SNAT2.REC
SU31040201_7_1_PCGe2.PAR
SU31040201_13_1_rsfmr.REC
SU31040201_12_1_DTIap.PAR
SU31040201_6_1_PCGe1.REC
SU31040201_7_1_PCGe2.REC
SU31040201_11_1_DTIpa.REC
SU31040201_11_1_DTIpa.PAR
SU31040201_9_1_T2str.PAR
SU31040201_5_1_SNAT3.PAR
SU31040201_4_1_SNAT2.PAR
SU31040201_13_1_rsfmr.PAR
SU31040201_9_1_T2str.REC
SU31040201_5_1_SNAT3.REC
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31081702
SU31081702_4_1_SNAT2.PAR
SU31081702_5_1_SNAT3.PAR
SU31081702_6_1_PCGe1.REC
SU31081702_8_1_T1mri.REC
SU31081702_11_1_DTIpa.REC
SU31081702_8_1_T1mri.PAR
SU31081702_13_1_rsfmr.REC
SU31081702_11_1_DTIpa.PAR
SU31081702_12_1_DTIap.PAR
SU31081702_10_1_Bzero.PAR
SU31081702_4_1_SNAT2.REC
SU31081702_13_1_rsfmr.PAR
SU31081702_7_1_PCGe2.PAR
SU3

/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31031902
SU31031902_8_1_T1mri.PAR
SU31031902_7_1_PCGe2.PAR
SU31031902_6_1_PCGe1.REC
SU31031902_10_1_Bzero.REC
SU31031902_10_1_Bzero.PAR
SU31031902_3_1_SNAT1.REC
SU31031902_11_1_DTIpa.REC
SU31031902_12_1_DTIap.PAR
SU31031902_5_1_SNAT3.REC
SU31031902_7_1_PCGe2.REC
SU31031902_6_1_PCGe1.PAR
SU31031902_5_1_SNAT3.PAR
SU31031902_11_1_DTIpa.PAR
SU31031902_9_1_T2str.REC
SU31031902_12_1_DTIap.REC
SU31031902_9_1_T2str.PAR
SU31031902_13_1_rsfmr.REC
SU31031902_13_1_rsfmr.PAR
SU31031902_4_1_SNAT2.REC
SU31031902_3_1_SNAT1.PAR
SU31031902_4_1_SNAT2.PAR
SU31031902_8_1_T1mri.REC
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31001302
SU31001302_3_1_SNAT1.PAR
SU31001302_10_1_Bzero.PAR
SU31001302_8_1_T1mri.REC
SU31001302_11_1_DTIpa.PAR
SU31001302_7_1_PCGe2.REC
SU31001302_5_1_SNAT3.PAR
SU31001302_9_1_T2str.REC
SU31001302_12_1_DTIap.PAR
SU31001302_3_1_SNAT1.REC
SU31001302_13_1_rsfmr.REC
SU31001302_4_1_SNAT2.REC
SU31001302_6_1_PCGe1.REC
SU31001302_13_1_rsfmr.PAR
SU31

/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31113101
SU31113101_9_1_T2str.PAR
SU31113101_6_1_PCGe1.REC
SU31113101_13_1_rsfmr.REC
SU31113101_13_1_rsfmr.PAR
SU31113101_10_1_Bzero.REC
SU31113101_6_1_PCGe1.PAR
SU31113101_5_1_SNAT3.REC
SU31113101_3_1_SNAT1.REC
SU31113101_8_1_T1mri.REC
SU31113101_4_1_SNAT2.REC
SU31113101_7_1_PCGe2.PAR
SU31113101_5_1_SNAT3.PAR
SU31113101_9_1_T2str.REC
SU31113101_11_1_DTIpa.REC
SU31113101_8_1_T1mri.PAR
SU31113101_3_1_SNAT1.PAR
SU31113101_7_1_PCGe2.REC
SU31113101_12_1_DTIap.PAR
SU31113101_11_1_DTIpa.PAR
SU31113101_12_1_DTIap.REC
SU31113101_10_1_Bzero.PAR
SU31113101_4_1_SNAT2.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_SCANS/SU31090201
SU31090201_12_1_DTIap.REC
SU31090201_10_1_Bzero.REC
SU31090201_4_1_SNAT2.PAR
SU31090201_6_1_PCGe1.REC
SU31090201_5_1_SNAT3.PAR
SU31090201_6_1_PCGe1.PAR
SU31090201_5_1_SNAT3.REC
SU31090201_8_1_T1mri.REC
SU31090201_9_1_T2str.PAR
SU31090201_9_1_T2str.REC
SU31090201_13_1_rsfmr.REC
SU31090201_3_1_SNAT1.REC
SU31090201_3_1_SNAT1.PAR
SU3109

## <font color = 'blue' >STEP 2: PSEUDOBIDS </font>
This step of this notebook will show you how to structure your data into pseudoBIDS format. 

We make the pseudobids dir (if it does not exist yet)  
Next, using this list we do the following:  
For every participant we make a new folder with different sessions underneath (which resembles the bids format). So previously we had SU31, SU33, SU35, we now call these ses-w01lab, ses-w03lab, ses-w05lab and they are nested within participants.  
After running the code, the structure looks like this:  

    /projectname/data/bidsify/pseudobids/
                                    ├── sub-mcc000101/
                                    |            ├── ses-w01lab/
                                    |            ├── ses-w03lab/
                                    |            └── ses-w05lab/
                                    ├── sub-mcc000102/
                                    |            ├── ses-w01lab/
                                    |            ├── ses-w03lab/
                                    |            └── ses-w05lab/
                                    ├── sub-mcc000201/
                                    |            ├── ses-w01lab/
                                    |            ├── ses-w03lab/
                                    |            └── ses-w05lab/ 
                                    └── sub-etcetera/
                                                 ├── ses-w01lab/
                                                 ├── ses-w03lab/
                                                 └── ses-w05lab/ 

**<font color = 'blue' >STEP 2.1: Pseudobids Directory </font>** 

You can use the code below to make a list with all your participant names in the rawDir. 

Assuming that your directories contain all the participant names, this can be easily done. 
Note that we strip the name and only use the number (to make it timepoint agnostic) 


In [16]:
dirlist2 = []
for root, directories, filenames in os.walk(rawDir):
    for directory in directories:
        if directory.startswith('SU'):
            dirlist2.append(directory[4:10]) 

print(dirlist2)

['049601', '066801', '064602', '012501', '026502', '011402', '004502', '028902', '098901', '096002', '000701', '095202', '113502', '076201', '040501', '041502', '030102', '085101', '074301', '077302', '015701', '112901', '018402', '043902', '046202', '064801', '076001', '064901', '017601', '013302', '059301', '102101', '020502', '001202', '081001', '024401', '022201', '056802', '109002', '034202', '056301', '071802', '049602', '090102', '046501', '000901', '020402', '083402', '088502', '037502', '115701', '090202', '079401', '098802', '100301', '050602', '012401', '083702', '053502', '067401', '110402', '091901', '107801', '113602', '088801', '087202', '013701', '064601', '033701', '094301', '018802', '015402', '059302', '064802', '009601', '003901', '021801', '031702', '098101', '046401', '083101', '034501', '026402', '094501', '028301', '092802', '080701', '109001', '072001', '072002', '083801', '081701', '024402', '046201', '103302', '053801', '085001', '021802', '081901', '020501',

In [17]:
# set prefix for middle childhood cohort
prefix = 'sub-mcc'
newp2 = Path(pseudoDir)

# make pseudoDir
if not Path.exists(newp2):
    Path(newp2).mkdir(parents = True)

# make participant folders
for folder in dirlist2:
    folder = folder.strip()
    if not Path.exists(newp2/(prefix + str(folder))/(str('ses-w01lab'))):
        Path(newp2/(prefix + str(folder))/(str('ses-w01lab'))).mkdir(parents=True)
        print(newp2/(prefix + str(folder))/(str('ses-w01lab')))
    if not Path.exists(newp2/(prefix + str(folder))/(str('ses-w03lab'))):
        Path(newp2/(prefix + str(folder))/(str('ses-w03lab'))).mkdir(parents=True)
        print(newp2/(prefix + str(folder))/(str('ses-w03lab')))
    if not Path.exists(newp2/(prefix + str(folder))/(str('ses-w05lab'))):
        Path(newp2/(prefix + str(folder))/(str('ses-w05lab'))).mkdir(parents=True)
        print(newp2/(prefix + str(folder))/(str('ses-w05lab')))

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049601/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049601/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049601/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc066801/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc066801/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc066801/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc064602/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc064602/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc064602/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc012501/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc012501/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc012501/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc115701/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc115701/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc115701/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090202/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090202/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090202/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc079401/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc079401/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc079401/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc098802/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc098802/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc098802/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc080701/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc080701/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc109001/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc109001/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc109001/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc072001/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc072001/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc072001/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc072002/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc072002/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc072002/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc083801/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc008101/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc111202/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc111202/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc111202/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc106702/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc106702/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc106702/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc041301/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc041301/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc041301/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc024201/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc024201/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc013601/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc014801/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc014801/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc014801/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc023501/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc023501/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc023501/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc012801/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc012801/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc012801/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc005101/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc005101/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobid

...

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc021501/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc021501/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc021501/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc007002/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc007002/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc007002/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc085202/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc085202/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc085202/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc080602/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc080602/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc080602/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc110201/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc026501/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc026501/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc026501/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc082201/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc082201/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc082201/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc067901/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc067901/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc067901/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc063602/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc063602/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc076502/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc076502/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc076502/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc089302/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc089302/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc089302/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc039701/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc039701/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc039701/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020102/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020102/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020102/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc007101/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc007101/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc096802/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc096802/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc096802/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc023502/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc023502/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc023502/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc047101/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc047101/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc047101/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc103301/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc113101/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc113101/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090201/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090201/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090201/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc006401/ses-w01lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc006401/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc006401/ses-w05lab


**<font color = 'blue' >STEP 2.2: PAR/REC to Pseudobids directory </font>** 

Now use the code below to place every PAR/REC file you want to convert to BIDS format, in the correct sub-directories (e.g. subject/ses-01 dir). 

Note that you have to specify the correct names of your anat scans and functional scans (for each run) below. You can do that by replacing the file names in red. Note that the file names should contain stars ('*') at the start and the end to make sure that the code runs for all participants (i.e. you only specify what the filenames of all participants have in common). 

- **Note**: In this step, we also add the 'bold' suffix to all bold scans. This will help bidsify to find the correct files later

In [ ]:
import shutil

prefix = 'sub-mcc'

for root, directories, filenames in os.walk(rawDir):
    if filenames != []:
        for file in filenames:
            #extract the file name and path
            name = root + '/' + file
            name = name.replace("\\", "/")
            dest_bold = os.path.join(pseudoDir, prefix + file[4:10] + '/' + 'ses-w01lab' + '/' + \
                                     file[-25:-4] + '_bold' + file[-4:])
            dest = os.path.join(pseudoDir, prefix + file[4:10] + '/' + 'ses-w01lab' + '/')


            if file[-9:-4] == 'PCGe1' or file[-9:-4] == 'PCGe2' \
            or file[-9:-4] == 'SNAT1' or file[-9:-4] == 'SNAT2' or file[-9:-4] == 'SNAT3'or file[-9:-4] == 'rsfmr':
                if not os.path.isfile(dest_bold):
                    print(dest_bold)
                    shutil.copy(name, dest_bold)
            
            elif file[-9:-4] == 'T1mri' or file[-9:-4] == 'T2str' or file[-9:-4] == 'Bzero' or file[-9:-4] == 'DTIap' or file[-9:-4] == 'DTIpa':
                if not os.path.isfile(dest):
                    print(dest)
                    shutil.copy(name, dest)
                else:
                    print('WARNING: file ' + dest + ' already exists in the folder! This file will therefore be skipped!')


/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049601/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049601/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049601/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049601/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049601/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049601/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049601/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049601/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049601/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049601/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc066801/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc066801/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc000701/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc000701/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc000701/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc000701/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc000701/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc000701/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc095202/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc095202/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc095202/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc095202/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc095202/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc095202/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc112901/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc112901/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc112901/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc018402/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc018402/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc018402/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc018402/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc018402/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc018402/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc018402/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc018402/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc018402/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020502/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020502/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020502/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020502/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020502/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020502/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020502/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020502/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020502/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc001202/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc001202/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc001202/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049602/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049602/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc049602/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090102/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090102/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090102/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090102/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090102/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090102/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090102/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090102/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090102/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc098802/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc098802/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc098802/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc098802/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc098802/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc100301/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc100301/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc100301/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc100301/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc100301/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc100301/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc100301/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bid

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc088801/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc088801/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc088801/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc088801/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc088801/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc088801/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc088801/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc088801/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc088801/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc087202/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc087202/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc087202/ses-w01lab/
/exports/fsw/Bendlab/SamenUniek/SU31_bid

**<font color = 'blue' >STEP 2.3: Check </font>** 
we can check the files in the pseudobids directories to see if this went well:

In [2]:
for root, directories, filenames in os.walk(pseudoDir):
    print(root) 
    for filename in filenames:
        print(filename)
        #break
    

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc059602
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc059602/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc059602/ses-w01lab
SU31059602_12_1_DTIap.PAR
SU31059602_10_1_Bzero.REC
SU31059602_11_1_DTIpa.REC
SU31059602_13_1_rsfmr_bold.REC
SU31059602_6_1_PCGe1_bold.REC
SU31059602_11_1_DTIpa.PAR
SU31059602_8_1_T1mri.PAR
SU31059602_5_1_SNAT3_bold.REC
SU31059602_10_1_Bzero.PAR
SU31059602_7_1_PCGe2_bold.PAR
SU31059602_7_1_PCGe2_bold.REC
SU31059602_4_1_SNAT2_bold.PAR
SU31059602_13_1_rsfmr_bold.PAR
SU31059602_3_1_SNAT1_bold.PAR
SU31059602_6_1_PCGe1_bold.PAR
SU31059602_5_1_SNAT3_bold.PAR
SU31059602_4_1_SNAT2_bold.REC
SU31059602_9_1_T2str.REC
SU31059602_3_1_SNAT1_bold.REC
SU31059602_8_1_T1mri.REC
SU31059602_12_1_DTIap.REC
SU31059602_9_1_T2str.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc059602/ses-w05lab
/exports/fsw/Bendlab/Sa

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc105902
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc105902/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc105902/ses-w01lab
SU31105902_11_1_T1mri.PAR
SU31105902_6_1_PCGe1_bold.REC
SU31105902_9_1_T2str.REC
SU31105902_5_1_SNAT3_bold.PAR
SU31105902_3_1_SNAT1_bold.REC
SU31105902_11_1_T1mri.REC
SU31105902_14_1_rsfmr_bold.REC
SU31105902_7_1_PCGe2_bold.PAR
SU31105902_10_1_Bzero.PAR
SU31105902_13_1_DTIap.REC
SU31105902_7_1_PCGe2_bold.REC
SU31105902_14_1_rsfmr_bold.PAR
SU31105902_5_1_SNAT3_bold.REC
SU31105902_6_1_PCGe1_bold.PAR
SU31105902_9_1_T2str.PAR
SU31105902_4_1_SNAT2_bold.PAR
SU31105902_12_1_DTIpa.PAR
SU31105902_10_1_Bzero.REC
SU31105902_12_1_DTIpa.REC
SU31105902_13_1_DTIap.PAR
SU31105902_4_1_SNAT2_bold.REC
SU31105902_3_1_SNAT1_bold.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc105902/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020501
/export

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc081002/ses-w01lab
SU31081002_13_1_DTIap.REC
SU31081002_4_1_SNAT2_bold.PAR
SU31081002_11_1_Bzero.REC
SU31081002_9_1_T1mri.PAR
SU31081002_6_1_SNAT3_bold.REC
SU31081002_6_1_SNAT3_bold.PAR
SU31081002_14_1_rsfmr_bold.REC
SU31081002_3_1_SNAT1_bold.REC
SU31081002_7_1_PCGe1_bold.REC
SU31081002_14_1_rsfmr_bold.PAR
SU31081002_10_1_T2str.REC
SU31081002_8_1_PCGe2_bold.REC
SU31081002_12_1_DTIpa.PAR
SU31081002_11_1_Bzero.PAR
SU31081002_12_1_DTIpa.REC
SU31081002_3_1_SNAT1_bold.PAR
SU31081002_10_1_T2str.PAR
SU31081002_7_1_PCGe1_bold.PAR
SU31081002_4_1_SNAT2_bold.REC
SU31081002_13_1_DTIap.PAR
SU31081002_9_1_T1mri.REC
SU31081002_8_1_PCGe2_bold.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc081002/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc059301
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc059301/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc059301/ses-w01

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090501/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090501/ses-w01lab
SU31090501_7_1_PCGe2_bold.PAR
SU31090501_11_1_DTIpa.REC
SU31090501_4_1_SNAT2_bold.REC
SU31090501_13_1_rsfmr_bold.REC
SU31090501_3_1_SNAT1_bold.REC
SU31090501_10_1_Bzero.PAR
SU31090501_6_1_PCGe1_bold.PAR
SU31090501_5_1_SNAT3_bold.PAR
SU31090501_11_1_DTIpa.PAR
SU31090501_13_1_rsfmr_bold.PAR
SU31090501_8_1_T1mri.REC
SU31090501_3_1_SNAT1_bold.PAR
SU31090501_6_1_PCGe1_bold.REC
SU31090501_5_1_SNAT3_bold.REC
SU31090501_8_1_T1mri.PAR
SU31090501_7_1_PCGe2_bold.REC
SU31090501_12_1_DTIap.PAR
SU31090501_9_1_T2str.REC
SU31090501_10_1_Bzero.REC
SU31090501_4_1_SNAT2_bold.PAR
SU31090501_9_1_T2str.PAR
SU31090501_12_1_DTIap.REC
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc090501/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc098802
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc098802/ses-w03la

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020001
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020001/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020001/ses-w01lab
SU31020001_3_1_SNAT1_bold.PAR
SU31020001_13_1_rsfmr_bold.REC
SU31020001_10_1_Bzero.REC
SU31020001_7_1_PCGe2_bold.PAR
SU31020001_9_1_T2str.REC
SU31020001_7_1_PCGe2_bold.REC
SU31020001_11_1_DTIpa.REC
SU31020001_6_1_PCGe1_bold.PAR
SU31020001_11_1_DTIpa.PAR
SU31020001_3_1_SNAT1_bold.REC
SU31020001_4_1_SNAT2_bold.PAR
SU31020001_8_1_T1mri.PAR
SU31020001_4_1_SNAT2_bold.REC
SU31020001_12_1_DTIap.PAR
SU31020001_12_1_DTIap.REC
SU31020001_9_1_T2str.PAR
SU31020001_13_1_rsfmr_bold.PAR
SU31020001_6_1_PCGe1_bold.REC
SU31020001_5_1_SNAT3_bold.PAR
SU31020001_8_1_T1mri.REC
SU31020001_10_1_Bzero.PAR
SU31020001_5_1_SNAT3_bold.REC
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc020001/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc081001
/exports/

...

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc079002/ses-w01lab
SU31079002_5_1_SNAT3_bold.REC
SU31079002_12_1_DTIap.REC
SU31079002_7_1_PCGe2_bold.PAR
SU31079002_3_1_SNAT1_bold.PAR
SU31079002_12_1_DTIap.PAR
SU31079002_5_1_SNAT3_bold.PAR
SU31079002_4_1_SNAT2_bold.REC
SU31079002_8_1_T1mri.PAR
SU31079002_6_1_PCGe1_bold.PAR
SU31079002_11_1_DTIpa.PAR
SU31079002_6_1_PCGe1_bold.REC
SU31079002_8_1_T1mri.REC
SU31079002_4_1_SNAT2_bold.PAR
SU31079002_11_1_DTIpa.REC
SU31079002_13_1_rsfmr_bold.PAR
SU31079002_10_1_Bzero.PAR
SU31079002_7_1_PCGe2_bold.REC
SU31079002_9_1_T2str.REC
SU31079002_3_1_SNAT1_bold.REC
SU31079002_9_1_T2str.PAR
SU31079002_10_1_Bzero.REC
SU31079002_13_1_rsfmr_bold.REC
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc079002/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc007801
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc007801/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc007801/ses-w01la

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc002102/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc076201
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc076201/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc076201/ses-w01lab
SU31076201_11_1_DTIpa.REC
SU31076201_13_1_rsfmr_bold.PAR
SU31076201_8_1_T1mri.PAR
SU31076201_11_1_DTIpa.PAR
SU31076201_5_1_SNAT3_bold.PAR
SU31076201_4_1_SNAT2_bold.REC
SU31076201_12_1_DTIap.PAR
SU31076201_3_1_SNAT1_bold.REC
SU31076201_3_1_SNAT1_bold.PAR
SU31076201_7_1_PCGe2_bold.PAR
SU31076201_13_1_rsfmr_bold.REC
SU31076201_6_1_PCGe1_bold.PAR
SU31076201_9_1_T2str.PAR
SU31076201_7_1_PCGe2_bold.REC
SU31076201_5_1_SNAT3_bold.REC
SU31076201_9_1_T2str.REC
SU31076201_8_1_T1mri.REC
SU31076201_10_1_Bzero.PAR
SU31076201_10_1_Bzero.REC
SU31076201_4_1_SNAT2_bold.PAR
SU31076201_6_1_PCGe1_bold.REC
SU31076201_12_1_DTIap.REC
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc076201/ses-w05la

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc000601/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc031701
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc031701/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc031701/ses-w01lab
SU31031701_3_1_SNAT1_bold.REC
SU31031701_4_1_SNAT2_bold.PAR
SU31031701_8_1_T1mri.REC
SU31031701_8_1_T1mri.PAR
SU31031701_5_1_SNAT3_bold.PAR
SU31031701_9_1_T2str.REC
SU31031701_9_1_T2str.PAR
SU31031701_6_1_PCGe1_bold.REC
SU31031701_11_1_DTIpa.REC
SU31031701_12_1_DTIap.PAR
SU31031701_4_1_SNAT2_bold.REC
SU31031701_6_1_PCGe1_bold.PAR
SU31031701_13_1_rsfmr_bold.REC
SU31031701_7_1_PCGe2_bold.REC
SU31031701_12_1_DTIap.REC
SU31031701_7_1_PCGe2_bold.PAR
SU31031701_11_1_DTIpa.PAR
SU31031701_10_1_Bzero.REC
SU31031701_5_1_SNAT3_bold.REC
SU31031701_3_1_SNAT1_bold.PAR
SU31031701_13_1_rsfmr_bold.PAR
SU31031701_10_1_Bzero.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc031701/ses-w05la

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc022302/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc006402
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc006402/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc006402/ses-w01lab
SU31006402_12_1_DTIap.REC
SU31006402_3_1_SNAT1_bold.PAR
SU31006402_10_1_Bzero.REC
SU31006402_3_1_SNAT1_bold.REC
SU31006402_5_1_SNAT3_bold.PAR
SU31006402_12_1_DTIap.PAR
SU31006402_8_1_T1mri.PAR
SU31006402_7_1_PCGe2_bold.PAR
SU31006402_9_1_T2str.PAR
SU31006402_8_1_T1mri.REC
SU31006402_5_1_SNAT3_bold.REC
SU31006402_4_1_SNAT2_bold.PAR
SU31006402_11_1_DTIpa.PAR
SU31006402_10_1_Bzero.PAR
SU31006402_7_1_PCGe2_bold.REC
SU31006402_11_1_DTIpa.REC
SU31006402_6_1_PCGe1_bold.PAR
SU31006402_4_1_SNAT2_bold.REC
SU31006402_6_1_PCGe1_bold.REC
SU31006402_9_1_T2str.REC
SU31006402_13_1_rsfmr_bold.REC
SU31006402_13_1_rsfmr_bold.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc006402/ses-w05la

/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc030101/ses-w03lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc030101/ses-w01lab
SU31030101_7_1_PCGe2_bold.REC
SU31030101_11_1_DTIpa.PAR
SU31030101_8_1_T1mri.REC
SU31030101_13_1_rsfmr_bold.REC
SU31030101_10_1_Bzero.PAR
SU31030101_12_1_DTIap.REC
SU31030101_3_1_SNAT1_bold.REC
SU31030101_10_1_Bzero.REC
SU31030101_9_1_T2str.REC
SU31030101_12_1_DTIap.PAR
SU31030101_6_1_PCGe1_bold.REC
SU31030101_13_1_rsfmr_bold.PAR
SU31030101_5_1_SNAT3_bold.PAR
SU31030101_9_1_T2str.PAR
SU31030101_5_1_SNAT3_bold.REC
SU31030101_3_1_SNAT1_bold.PAR
SU31030101_6_1_PCGe1_bold.PAR
SU31030101_4_1_SNAT2_bold.REC
SU31030101_11_1_DTIpa.REC
SU31030101_8_1_T1mri.PAR
SU31030101_4_1_SNAT2_bold.PAR
SU31030101_7_1_PCGe2_bold.PAR
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc030101/ses-w05lab
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc033702
/exports/fsw/Bendlab/SamenUniek/SU31_bids2/pseudobids/sub-mcc033702/ses-w03la

## <font color = 'blue' > STEP 3: BIDS </font>

We now use the bidsify tool (https://github.com/NILAB-UvA/bidsify) created by Lukas Snoek to convert the pseudoBIDS data into BIDS. 

All kudos should go to the bidsify creators, this is just a guide that allows the usage of bidsify with an interactive notebook. All kudos for this notebook should go to Eduard Klapwijk, who originally wrote the manual for Braintime data. Michelle Achterberg made adjustments for the LCID MCC data. 

For more information on BIDS visit https://bids.neuroimaging.io and read the Gorgolewski et al. paper about the BIDS format here: https://doi.org/10.1038/sdata.2016.44
Another useful resource is the BIDS starter kit: https://github.com/bids-standard/bids-starter-kit

There are multiple tools out there to convert datasets into Nifti (one of the main ones being Heudiconv: https://github.com/nipy/heudiconv). We use bidsify because this converter is capable of working with PAR/REC files.

**<font color = 'blue' >STEP 3.1: install bidsify</font>**   
Install bidsify + most dependencies
- see for more info https://github.com/NILAB-UvA/bidsify)

<u>This is what we have to do:</u>
1. Install bidsify:
pip install bidsify
2. Install / check dependencies:
dcm2niix (release v1.0.20181125 or newer)
nibabel
scipy
numpy
joblib (for parallelization)
pandas
3. Make a config-file in either the json or YAML format. 

In [14]:
# Install bidsify package in the current Jupyter kernel
!pip install bidsify

You are using pip version 9.0.3, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


**<font color = 'blue' >STEP 3.2:  Install dcm2niix </font>**

On local machine install using Conda:

In [12]:
!conda install --yes -c conda-forge dcm2niix 

Solving environment: done

# All requested packages already installed.



On Shark HPC do the following in your user or user/jupyter folder:

In [ ]:
#### installing dcm2niix on Shark:
cd /home/jupyter/bin
git clone https://github.com/rordenlab/dcm2niix.git
cd dcm2niix
mkdir build && cd build
# to avoid the error when using make .., do: 
cmake -DUSE_STATIC_RUNTIME=OFF ..
make
# finally copy binary to user/bin:
cp dcmniix etklapwijk/bin

**<font color = 'blue' >STEP 3.3: install bids-validator</font>**

In [15]:
pip install bids_validator

You are using pip version 9.0.3, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


**<font color = 'blue' >STEP 3.4:  Make a config-file </font>** 

In order to run the bidsify script we have to make a config-file in either the json or YAML format. This file contains information about the experiment, such as the types and names of the anatomical and functional scans. The bidsify script needs this information to strucure the data according to the BIDS format. You will have to create this json file yourself as it is specific for each functional MRI task. You can find an example here: https://github.com/eur-synclab/bidsification/blob/master/config.json.

We'll go for json. We can either use R package 'jsonlite' or use http://jsoneditoronline.org. See https://github.com/NILAB-UvA/bidsify#the-config-file for details.
Place the file in the current directory --> see example ./config.json

Below you can find the text of the L-CID MCC wave 01 lab config file. 

options: 
   mri_ext: PAR 
   debug: true
   subject_stem: sub
   deface: true
   out_dir: bids

mappings:
  bold: bold
  T1w: T1mri
  T2w: T2str
  dwi: DTI
  phasediff: Bzero
          
metadata:
  MagneticFieldStrength: 3
  InstitutionName: Leiden Consortium on Individual Development, Leiden University, Leiden, The Netherlands

func: 
  SNAT-run1: 
    id: SNAT1  #corresponds to the name in pseudobids
    task: snat #the bids name
    run: 1
          
  SNAT-run2:
    id: SNAT2
    task: snat
    run: 2
           
  SNAT-run3:
    id: SNAT3
    task: snat
    run: 3
    
  PCG-run1: 
   id: PCGe1
   task: pcgex
   run: 1
              
  PCG-run2: 
   id: PCGe2
   task: pcgex
   run: 2
  
  Restingstate:
   id: rsfmr
   task: rest
            
anat:
  T1:
   id: T1mri

  T2:
   id: T2str
   
fmap:
  B0:
   id: Bzero

dwi:
  DTIset1:
   id: DTIap
   run: 1
   acq: 
   
  

In [17]:
with open('/exports/fsw/Bendlab/SamenUniek/SU31_bids2/config.json', 'r') as config:
    print(config.read())

{
    "options":  {
          "mri_ext": "PAR",
          "debug": true,
          "subject_stem": "sub",
          "deface": false,
          "spinoza_data": false,
          "out_dir": "final"
        },
      "mappings": {
          "bold": "bold",
          "T1w": "T1mri",
          "dwi": "DTI",
          "phasediff": "Bzero",
          "T2w": "T2str"
        },
      "metadata": {
          "MagneticFieldStrength": 3,
          "InstitutionName": "Leiden Consortium on Individual Development, Leiden University, Leiden, The Netherlands"
        },
      "func": {
        "SNAT-run1": {
            "id": "SNAT1",
            "task": "snat",
            "run": 1
          },
          "SNAT-run2":{
              "id": "SNAT2",
              "task": "snat",
              "run": 2
            },
          "SNAT-run3":{
              "id": "SNAT3",
              "task": "snat",
              "run": 3
            },
          "PCGex-run1": {
              "id": "PCGe1",
              "ta

**<font color = 'blue' >STEP 3.5:  Run Bidsify </font>** 

Run Bidsify command to turn data into BIDS format
see https://github.com/NILAB-UvA/bidsify#how-does-it-work for details

To run the bidsify script, you have to replace the path to your .json file and pseudobids directory below with the path to your own json file and pseudobids directory. 

In [10]:
#note1: adjust the path to the config.json file 
#note2: adjust the path to the psuedobids directory

!bidsify -c /exports/fsw/Bendlab/SamenUniek/SU31_bids/config.json -d /exports/fsw/Bendlab/SamenUniek/SU31_bids/pseudobids

Setting output-dir to /exports/fsw/Bendlab/SamenUniek/SU31_bids/bids
Running bidsify with the following arguments:
	 directory=/exports/fsw/Bendlab/SamenUniek/SU31_bids/pseudobids 
	 config=/exports/fsw/Bendlab/SamenUniek/SU31_bids/config.json 
	 out_dir=/exports/fsw/Bendlab/SamenUniek/SU31_bids/bids 
	 validate=False

/home/etklapwijk/jupyter/lib64/python3.6/site-packages/bidsify/main.py:371: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(config)
which: no bids-validator in (/home/etklapwijk/jupyter/bin:/home/etklapwijk/bin:/share/software/bin/:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/dell/srvadmin/iSM/bin)
Data from sub-mcc000601 has been converted already - skipping ...
Data from sub-mcc000601 has been converted already - skipping ...
Data from sub-mcc000601 has been converted already - skipping ...
Data from sub-mcc000701 has been conver

Data from sub-mcc007801 has been converted already - skipping ...
Data from sub-mcc007801 has been converted already - skipping ...
Data from sub-mcc007802 has been converted already - skipping ...
Data from sub-mcc007802 has been converted already - skipping ...
Data from sub-mcc007802 has been converted already - skipping ...
Data from sub-mcc007901 has been converted already - skipping ...
Data from sub-mcc007901 has been converted already - skipping ...
Data from sub-mcc007901 has been converted already - skipping ...
Data from sub-mcc007902 has been converted already - skipping ...
Data from sub-mcc007902 has been converted already - skipping ...
Data from sub-mcc007902 has been converted already - skipping ...
Data from sub-mcc008101 has been converted already - skipping ...
Data from sub-mcc008101 has been converted already - skipping ...
Data from sub-mcc008101 has been converted already - skipping ...
Data from sub-mcc008102 has been converted already - skipping ...
Data from 

Data from sub-mcc016302 has been converted already - skipping ...
Data from sub-mcc016302 has been converted already - skipping ...
Data from sub-mcc016302 has been converted already - skipping ...
Data from sub-mcc016701 has been converted already - skipping ...
Data from sub-mcc016701 has been converted already - skipping ...
Data from sub-mcc016701 has been converted already - skipping ...
Data from sub-mcc016901 has been converted already - skipping ...
Data from sub-mcc016901 has been converted already - skipping ...
Data from sub-mcc016901 has been converted already - skipping ...
Data from sub-mcc016902 has been converted already - skipping ...
Data from sub-mcc016902 has been converted already - skipping ...
Data from sub-mcc016902 has been converted already - skipping ...
Data from sub-mcc017402 has been converted already - skipping ...
Data from sub-mcc017402 has been converted already - skipping ...
Data from sub-mcc017402 has been converted already - skipping ...
Data from 

Data from sub-mcc024401 has been converted already - skipping ...
Data from sub-mcc024401 has been converted already - skipping ...
Data from sub-mcc024402 has been converted already - skipping ...
Data from sub-mcc024402 has been converted already - skipping ...
Data from sub-mcc024402 has been converted already - skipping ...
Data from sub-mcc024801 has been converted already - skipping ...
Data from sub-mcc024801 has been converted already - skipping ...
Data from sub-mcc024801 has been converted already - skipping ...
Data from sub-mcc024802 has been converted already - skipping ...
Data from sub-mcc024802 has been converted already - skipping ...
Data from sub-mcc024802 has been converted already - skipping ...
Data from sub-mcc025101 has been converted already - skipping ...
Data from sub-mcc025101 has been converted already - skipping ...
Data from sub-mcc025101 has been converted already - skipping ...
Data from sub-mcc025102 has been converted already - skipping ...
Data from 

Data from sub-mcc037701 has been converted already - skipping ...
Data from sub-mcc037701 has been converted already - skipping ...
Data from sub-mcc037701 has been converted already - skipping ...
Data from sub-mcc037702 has been converted already - skipping ...
Data from sub-mcc037702 has been converted already - skipping ...
Data from sub-mcc037702 has been converted already - skipping ...
Data from sub-mcc037801 has been converted already - skipping ...
Data from sub-mcc037801 has been converted already - skipping ...
Data from sub-mcc037801 has been converted already - skipping ...
Data from sub-mcc037802 has been converted already - skipping ...
Data from sub-mcc037802 has been converted already - skipping ...
Data from sub-mcc037802 has been converted already - skipping ...
Data from sub-mcc038701 has been converted already - skipping ...
Data from sub-mcc038701 has been converted already - skipping ...
Data from sub-mcc038701 has been converted already - skipping ...
Data from 

...

Data from sub-mcc086701 has been converted already - skipping ...
Data from sub-mcc086701 has been converted already - skipping ...
Data from sub-mcc086702 has been converted already - skipping ...
Data from sub-mcc086702 has been converted already - skipping ...
Data from sub-mcc086702 has been converted already - skipping ...
Data from sub-mcc087201 has been converted already - skipping ...
Data from sub-mcc087201 has been converted already - skipping ...
Data from sub-mcc087201 has been converted already - skipping ...
Data from sub-mcc087202 has been converted already - skipping ...
Data from sub-mcc087202 has been converted already - skipping ...
Data from sub-mcc087202 has been converted already - skipping ...
Data from sub-mcc087701 has been converted already - skipping ...
Data from sub-mcc087701 has been converted already - skipping ...
Data from sub-mcc087701 has been converted already - skipping ...
Data from sub-mcc088401 has been converted already - skipping ...
Data from 

Data from sub-mcc095402 has been converted already - skipping ...
Data from sub-mcc095402 has been converted already - skipping ...
Data from sub-mcc095502 has been converted already - skipping ...
Data from sub-mcc095502 has been converted already - skipping ...
Data from sub-mcc095502 has been converted already - skipping ...
Data from sub-mcc095901 has been converted already - skipping ...
Data from sub-mcc095901 has been converted already - skipping ...
Data from sub-mcc095901 has been converted already - skipping ...
Data from sub-mcc095902 has been converted already - skipping ...
Data from sub-mcc095902 has been converted already - skipping ...
Data from sub-mcc095902 has been converted already - skipping ...
Data from sub-mcc096001 has been converted already - skipping ...
Data from sub-mcc096001 has been converted already - skipping ...
Data from sub-mcc096001 has been converted already - skipping ...
Data from sub-mcc096002 has been converted already - skipping ...
Data from 

Data from sub-mcc107101 has been converted already - skipping ...
Data from sub-mcc107102 has been converted already - skipping ...
Data from sub-mcc107102 has been converted already - skipping ...
Data from sub-mcc107102 has been converted already - skipping ...
Data from sub-mcc107801 has been converted already - skipping ...
Data from sub-mcc107801 has been converted already - skipping ...
Data from sub-mcc107801 has been converted already - skipping ...
Data from sub-mcc107802 has been converted already - skipping ...
Data from sub-mcc107802 has been converted already - skipping ...
Data from sub-mcc107802 has been converted already - skipping ...
Data from sub-mcc108101 has been converted already - skipping ...
Data from sub-mcc108101 has been converted already - skipping ...
Data from sub-mcc108101 has been converted already - skipping ...
Data from sub-mcc108102 has been converted already - skipping ...
Data from sub-mcc108102 has been converted already - skipping ...
Data from 

Philips Scaling Values RS:RI:SS = 0.74115:0:0.109654 (see PMC3998685)
Convert 1 DICOM as /exports/fsw/Bendlab/SamenUniek/SU31_bids/bids/sub-mcc113102/ses-w01lab/SU31113102_17_1_rsfmr_bold (80x80x37x140)
Conversion required 0.924726 seconds (0.312802 for core code).
Chris Rorden's dcm2niiX version v1.0.20200331  (JP2:OpenJPEG) (JP-LS:CharLS) GCC8.3.1 (64-bit Linux)
Done reading PAR header version 4.2, with 5254 slices
no Varies
0.78462 0 0.0760702
Philips Scaling Values RS:RI:SS = 0.78462:0:0.0760702 (see PMC3998685)
Convert 1 DICOM as /exports/fsw/Bendlab/SamenUniek/SU31_bids/bids/sub-mcc113102/ses-w01lab/SU31113102_4_1_SNAT2_bold (80x80x37x142)
Conversion required 1.016006 seconds (0.339986 for core code).
Chris Rorden's dcm2niiX version v1.0.20200331  (JP2:OpenJPEG) (JP-LS:CharLS) GCC8.3.1 (64-bit Linux)
Done reading PAR header version 4.2, with 2325 slices
no Varies
6.46886 0 0.0231212
Philips Scaling Values RS:RI:SS = 6.46886:0:0.0231212 (see PMC3998685)
Convert 1 DICOM as /exports

Data from sub-mcc113601 has been converted already - skipping ...
Data from sub-mcc113601 has been converted already - skipping ...
Data from sub-mcc113601 has been converted already - skipping ...
Data from sub-mcc113602 has been converted already - skipping ...
Data from sub-mcc113602 has been converted already - skipping ...
Data from sub-mcc113602 has been converted already - skipping ...
Data from sub-mcc113901 has been converted already - skipping ...
Data from sub-mcc113901 has been converted already - skipping ...
Data from sub-mcc113901 has been converted already - skipping ...
Data from sub-mcc113902 has been converted already - skipping ...
Data from sub-mcc113902 has been converted already - skipping ...
Data from sub-mcc113902 has been converted already - skipping ...
Data from sub-mcc115701 has been converted already - skipping ...
Data from sub-mcc115701 has been converted already - skipping ...
Data from sub-mcc115701 has been converted already - skipping ...
Data from 

We rename the T2w to T2star (for more clarity, and according to bids specifications)
In the bidsify package there is currently no T2star mapping (as far as I am aware), therefore we simply rename the files in this step. 

In [25]:
for root, directories, filenames in os.walk(base_dir + '/bids'):
    if filenames != []:
        for file in filenames:
            #extract the file name and path
            name = root + '/' + file
            name = name.replace("\\", "/")

            if 'T2w.nii' in file:
                renameT2nii = root + '/' + file[:-7] + 'T2star' + file[-4:]
                if not os.path.isfile(renameT2nii):
                    print('renamenii:' + renameT2nii)
                    os.rename(name, renameT2nii)
                else:
                    print('WARNING: file ' + renameT2nii + ' already exists in the folder! This file will therefore be skipped!')
            if 'T2w.json' in file:
                renameT2js = root + '/' + file[:-8] + 'T2star' + file[-5:]
                if not os.path.isfile(renameT2js):
                    print('renamejs:' + renameT2js)
                    os.rename(name, renameT2js)
                else:
                    print('WARNING: file ' + renameT2js + ' already exists in the folder! This file will therefore be skipped!')                                                         

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [18]:
import os
renamefmap = 'hi'

for root, directories, filenames in os.walk('/exports/fsw/Bendlab/SamenUniek/SU31_bids/data'):
    #if directories != []:
        for directory in directories:
            #extract the file name and path
            name = root + '/' + directory
            #print(name)

            if 'fmap' in directory:
                renamefmap = root + '/' + '.' + directory
                print('orig name:' + name)
                print('renamenii:' + renamefmap)
                os.rename(name, renamefmap)
                #else:
                 #   print('WARNING: file ' + renamefmap + ' already exists in the folder! This file will therefore be skipped!')
                                                                    

orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc059602/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc059602/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc067902/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc067902/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc046402/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc046402/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc044401/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc044401/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc084502/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc084502/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc089302/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUn

orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc043401/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc043401/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc093202/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc093202/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc041501/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc041501/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc033701/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc033701/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc108101/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc108101/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc107801/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUn

orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc053502/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc053502/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc053301/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc053301/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc106902/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc106902/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc034202/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc034202/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc076202/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc076202/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc102501/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUn

orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc083702/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc083702/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc014802/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc014802/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc036901/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc036901/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc040502/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc040502/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc038702/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc038702/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc103302/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUn

orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc089301/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc089301/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc044901/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc044901/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc011401/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc011401/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc041302/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc041302/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc019301/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc019301/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc096001/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUn

orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc088502/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc088502/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc013502/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc013502/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc046202/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc046202/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc092101/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc092101/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc037502/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc037502/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc061801/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUn

orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc028302/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc028302/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc098901/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc098901/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc006401/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc006401/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc007401/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc007401/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc088701/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc088701/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc028902/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUn

orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc008101/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc008101/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc117101/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc117101/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc002102/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc002102/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc076201/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc076201/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc005102/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc005102/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc065602/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUn

orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc003901/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc003901/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc011202/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc011202/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc037801/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc037801/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc052202/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc052202/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc001302/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc001302/ses-w01lab/.fmap
orig name:/exports/fsw/Bendlab/SamenUniek/SU31_bids/data/sub-mcc042601/ses-w01lab/fmap
renamenii:/exports/fsw/Bendlab/SamenUn

**<font color = 'blue' >STEP 3.6:  Check  </font>**  
If everything went well, we now have a new directory called 'bids' with our data in BIDS format
Let's have a look:

In [ ]:
for root, directories, filenames in os.walk('/exports/fsw/Bendlab/SamenUniek/SU25_SCANS/bids'):
    for directory in directories:
        print(directory)
    for filename in filenames:
        print(filename)     

**<font color = 'blue' >STEP 3.7:  BIDS validator  </font>**  
For now, the online BIDS validator can be used (https://bids-standard.github.io/bids-validator/) using Chrome or Firefox 


## Step 4: deface T1w data

In [3]:
pip install pydeface

    100% |████████████████████████████████| 8.2MB 102kB/s eta 0:00:01██████▊                         | 1.7MB 50.0MB/s eta 0:00:01 3.0MB 48.3MB/s eta 0:00:01████████▌             | 4.7MB 48.9MB/s eta 0:00:01
    100% |████████████████████████████████| 3.1MB 300kB/s eta 0:00:01��                           | 419kB 48.4MB/s eta 0:00:019MB/s eta 0:00:01
    100% |████████████████████████████████| 26.1MB 48kB/s  eta 0:00:01                      | 2.6MB 49.2MB/s eta 0:00:01           | 5.0MB 50.5MB/s eta 0:00:01              | 6.9MB 49.6MB/s eta 0:00:01K    33% |██████████▉                     | 8.8MB 51.5MB/s eta 0:00:01��█████▉                 | 12.0MB 15.2MB/s eta 0:00:01MB/s eta 0:00:01�██████████████▍           | 16.6MB 54.2MB/s eta 0:00:01�███▊        | 19.3MB 53.5MB/s eta 0:00:01.1MB/s eta 0:00:01/s eta 0:00:01
    100% |████████████████████████████████| 450kB 1.1MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 732kB/s eta 0:00:01
    100% |████████████████████████████

  Running setup.py install for traits ... done
  Running setup.py install for simplejson ... done
  Running setup.py install for pydotplus ... done
  Running setup.py install for pydeface ... done
You are using pip version 9.0.3, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
#change to testdir
os.chdir('/exports/fsw/Bendlab/SamenUniek/test_deface/')

In [7]:
# Loop through subjects and deface anats
with open('participants.tsv') as f:
    tsv = read(f)

for subject in tsv:
    
    chdir(join(bids_dir, subject, 'anat'))
    anat_fns = glob('*T*w.nii.gz')
    for anat_fn in anat_fns:
        
        #call(f'pydeface --outfile {anat_fn} --force {anat_fn}',
         #    shell=True)
        print(f"Finished defacing {anat_fn}!")

None


In [3]:
pip install bids_validator

You are using pip version 9.0.3, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from bids_validator import BIDSValidator
BIDSValidator().is_bids('/exports/fsw/Bendlab/SamenUniek/SU31_bids/data')

False